# Description

In this notebook we predict drug-disease associations using S-PrediXcan results and LINCS expression profiles.

See more details in notebook `011-prediction-gene_module_based.ipynb`.

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [4]:
PREDICTION_METHOD = "Gene-based"

In [5]:
LINCS_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "lincs"
display(LINCS_DATA_DIR)
assert LINCS_DATA_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs')

In [6]:
SPREDIXCAN_DATA_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"] / "spredixcan" / "raw"
display(SPREDIXCAN_DATA_DIR)
assert SPREDIXCAN_DATA_DIR.exists()

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/spredixcan/raw')

In [7]:
OUTPUT_PREDICTIONS_DIR = Path(LINCS_DATA_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/predictions')

# Load PharmacotherapyDB gold standard

In [8]:
gold_standard = pd.read_pickle(
    Path(conf.RESULTS["DRUG_DISEASE_ANALYSES"], "gold_standard.pkl"),
)

In [9]:
display(gold_standard.shape)

(998, 3)

In [10]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [11]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Raw data

In [12]:
input_file = Path(LINCS_DATA_DIR, "lincs-data.pkl").resolve()

display(input_file)

PosixPath('/home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/lincs-data.pkl')

In [13]:
lincs_projection = pd.read_pickle(input_file)

In [14]:
display(lincs_projection.shape)

(7120, 1170)

In [15]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
ENSG00000196839,-1.001,-1.835,1.391,1.132,0.257,1.932,0.508,1.408,0.777,0.032,...,-1.692,-0.516,-1.435,-0.317,-0.012,0.641,-0.230,-0.518,-0.177,2.146
ENSG00000170558,1.146,-1.863,0.011,-1.020,1.143,-0.115,1.327,0.310,-1.853,0.872,...,0.354,0.498,0.268,-1.084,-0.142,-0.077,0.633,-1.807,0.032,0.135
ENSG00000117020,-0.693,1.694,-0.804,-0.164,1.145,-1.465,1.221,-0.747,0.829,-0.961,...,-1.196,-0.230,-1.049,-0.347,0.586,0.865,-0.021,2.180,-0.956,0.105
ENSG00000133997,-0.037,0.383,0.269,-0.997,0.185,-0.536,0.424,-0.119,-1.313,0.579,...,-0.343,0.116,-0.245,-0.127,-1.367,0.149,0.117,2.084,1.178,0.772
ENSG00000101473,0.162,-0.899,0.105,-0.090,-1.291,1.404,0.185,0.157,-0.327,-0.026,...,-0.136,-1.115,-0.280,0.200,0.638,-0.197,-0.360,-2.302,-0.117,-0.167


# Load S-PrediXcan

In [16]:
phenomexcan_input_file_list = [
    f for f in SPREDIXCAN_DATA_DIR.glob("*.pkl") if f.name.startswith("spredixcan-")
]

In [17]:
display(len(phenomexcan_input_file_list))

49

In [18]:
pd.read_pickle(phenomexcan_input_file_list[0]).head()

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
gene_id,,,,,,,,,,,,,,,,,,,,,
ENSG00000000457,-1.452209,-0.802326,-1.561402,3.147679,-2.157754,-0.755943,-0.935751,-1.122251,0.032453,0.574413,...,-0.397510,1.633133,0.147963,0.566545,-0.671365,0.225834,-0.457130,-1.190160,-0.073451,-0.396977
ENSG00000000460,3.081034,0.855978,-1.924325,-1.669142,0.009112,0.159532,-0.401107,-0.338919,1.462722,-1.034109,...,1.126104,0.234430,-0.396063,-0.387089,-0.715332,-1.403748,-0.601588,1.278790,-1.646919,-1.092787
ENSG00000000938,-1.070723,0.517722,-0.803752,1.161886,0.315695,-0.519783,-1.221793,0.902990,-1.749067,-0.997415,...,-0.752216,0.074812,0.104969,0.836082,0.662738,1.525290,-0.478367,-0.424205,-0.296353,2.448133
ENSG00000000971,2.221978,-0.029211,-0.082311,-0.707300,0.860528,-0.435167,0.210884,1.454587,1.331105,-0.155145,...,3.271953,-0.805236,-1.050279,-0.311062,-0.266324,-0.961981,1.025476,1.432934,1.262197,-0.042457
ENSG00000001036,-1.291800,0.997475,-0.612067,-0.706709,-1.277902,-1.202425,0.818588,0.654576,0.316873,-0.599414,...,-2.058716,0.038065,0.497963,0.825052,0.676713,0.124185,-0.093519,2.580621,-0.589285,-1.509423


# Predict drug-disease associations

In [19]:
from drug_disease import predict_dotprod_neg

In [20]:
methods_to_run = [predict_dotprod_neg]

In [21]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)

    # get common genes with lincs
    common_genes = phenomexcan_projection.index.intersection(lincs_projection.index)
    phenomexcan_projection = phenomexcan_projection.loc[common_genes]
    lincs_projection = lincs_projection.loc[common_genes]

    print(f"  shape: {phenomexcan_projection.shape}")

    for prediction_method in methods_to_run:
        # here I use different thresholds to select the genes: the top 50, 100, 250 and 500, or all (None)
        for ntc in (None, 50, 100, 250, 500):
            prediction_method(
                lincs_projection,
                phenomexcan_input_file,
                phenomexcan_projection,
                OUTPUT_PREDICTIONS_DIR,
                PREDICTION_METHOD,
                doids_in_gold_standard,
                FORCE_RUN,
                n_top_conditions=ntc,
                use_abs=True,
            )

            print("\n")

spredixcan-mashr-zscores-Whole_Blood-data.pkl
  shape: (5544, 4091)
  predicting all_genes...
    shape: (1170, 4091)
  saving to: /home/miltondp/projects/labs/greenelab/phenoplier/base/results/drug_disease_analyses/lincs/predictions/dotprod_neg/spredixcan-mashr-zscores-Whole_Blood-data-all_genes-prediction_scores.h5
    saving full predictions...
    shape: (4786470, 3)


,score
count,4.786470e+06
mean,1.227732e+00
std,1.703457e+02
min,-1.719471e+04
25%,-5.016653e+01
50%,-1.177876e-01
75%,5.029694e+01
max,1.046353e+04


    key: full_prediction


/mnt/software/miniconda3/envs/phenoplier2/lib/python3.8/site-packages/tables/file.py:426: UserWarning: a closed node found in the registry: ``/full_prediction/meta/values_block_2/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


    saving classifier data...
      shape (after DOID map): (1170, 351)
    shape: (410670, 3)


...

    key: full_prediction


/mnt/software/miniconda3/envs/phenoplier2/lib/python3.8/site-packages/tables/file.py:426: UserWarning: a closed node found in the registry: ``/full_prediction/meta/values_block_2/meta/_i_table``
  warnings.warn("a closed node found in the registry: "


    saving classifier data...
      shape (after DOID map): (1170, 351)
    shape: (410670, 3)


,score
count,410670.000000
mean,9.127958
std,66.971988
min,-2796.329999
25%,-12.383217
50%,6.744847
75%,26.261810
max,3863.144000


    key: prediction


